In [43]:
import pandas as pd
import numpy as np
import quandl
import intrinio_sdk
import os

quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = os.getenv('INTRINO_API_KEY')

In [67]:
columns = ['calendardate', 'datekey', 'eps', 'currentratio', 'assetsnc', 'liabilitiesnc', 'cashneq', 
           'investments', 'investmentsc']
df = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker='TRV', qopts={"columns":columns})

In [68]:
df.head()

,calendardate,datekey,eps,currentratio,assetsnc,liabilitiesnc,cashneq,investments,investmentsc
None,,,,,,,,,
0,2017-12-31,2017-12-31,7.39,None,None,None,344000000.0,7.250200e+10,None
1,2016-12-31,2016-12-31,10.39,None,None,None,307000000.0,7.048800e+10,None
2,2015-12-31,2015-12-31,10.99,None,None,None,380000000.0,7.047000e+10,None
3,2014-12-31,2014-12-31,10.82,None,None,None,374000000.0,7.326100e+10,None
4,2013-12-31,2013-12-31,9.84,None,None,None,294000000.0,7.316000e+10,None


In [55]:
# Intrino
def get_historical_data(identifier, tag, frequency='yearly'):
    company_api = intrinio_sdk.CompanyApi()
    api_response = company_api.get_company_historical_data(identifier=identifier,
                                                           tag=tag,
                                                           frequency=frequency)
    data = api_response.historical_data
    indexes = []
    values = []
    for el in data:
        indexes.append(el.date)
        values.append(el.value)
    df = pd.DataFrame(data=values, index=indexes, columns=[tag])
    return df

In [60]:
df2 = get_historical_data(identifier='TRV', tag='totalcurrentliabilities', frequency='yearly')
df2.head()

,totalcurrentliabilities


Current ratio está faltando para algumas empresas nas bases da **Quandl** e da **Intrino**. O teste que estou usando é a **TRV**. Vamos tentar puxar no Yahoo Finance programaticamente.
## Pandas Data Reader

In [70]:
import pandas_datareader as pdr
from datetime import datetime

appl = pdr.get_data_yahoo(symbols='AAPL', start=datetime(2000, 1, 1), end=datetime(2012, 1, 1))
appl.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,4.017857,3.631696,3.745536,3.997768,133949200.0,2.677157
2000-01-04,3.950893,3.613839,3.866071,3.660714,128094400.0,2.451444
2000-01-05,3.948661,3.678571,3.705357,3.714286,194580400.0,2.487319
2000-01-06,3.821429,3.392857,3.790179,3.392857,191993200.0,2.272070
2000-01-07,3.607143,3.410714,3.446429,3.553571,115183600.0,2.379695


Pandas datareader is mostly deprecated. Let's check **IEX**.
## IEX Finance
[API documentation](https://iextrading.com/developer/docs/#financials)

[IEXFinance Python interface documentation](https://addisonlynch.github.io/iexfinance/stable/stocks.html#overview)

In [78]:
from iexfinance.stocks import Stock

aapl = Stock("TRV", output_format='pandas')

aapl.get_price()

,TRV
TRV,125.04


In [79]:
aapl.get_financials()

,cashChange,cashFlow,costOfRevenue,currentAssets,currentCash,currentDebt,grossProfit,netIncome,operatingExpense,operatingGainsLosses,operatingIncome,operatingRevenue,researchAndDevelopment,shareholderEquity,totalAssets,totalCash,totalDebt,totalLiabilities,totalRevenue
reportDate,,,,,,,,,,,,,,,,,,,
2018-09-30,-56000000,1734000000,None,None,359000000,600000000,None,709000000,None,-144000000,None,None,None,22460000000,104390000000,None,6564000000,81930000000,7723000000
2018-06-30,24000000,1144000000,None,None,415000000,500000000,None,524000000,None,-110000000,None,None,None,22623000000,103523000000,None,6464000000,80900000000,7477000000
2018-03-31,51000000,554000000,None,None,397000000,500000000,None,669000000,None,-84000000,None,None,None,22979000000,103676000000,None,6963000000,80697000000,7286000000
2017-12-31,-35000000,277000000,None,None,344000000,600000000,None,551000000,None,-167000000,None,None,None,23731000000,103483000000,None,6571000000,79752000000,7451000000


No data available as well... Let's drop it for now.